# Polarsで「pandasライブラリ活用入門(第2版)」を書いてみました
# はじめに
Pythonにおけるデータ分析ライブラリといえば[pandas](https://pandas.pydata.org/docs/reference/index.html)ですが、最近Rust言語で書かれた高速データ分析ライブラリ[polars](https://pola-rs.github.io/polars/py-polars/html/reference/)というライブラリへの注目が集まっています。
今年は[Qiitaのアドベントカレンダー](https://qiita.com/advent-calendar/2023/polars)も作成されているので、多くの方が気になるライブラリだと言えるでしょう。

私も今年の3月に[pandasとpolarsのコード比較記事](https://zenn.dev/tremendous1192/articles/bd1454621d624a)を執筆しましたが、X(旧Twitter)にて動かないコードがあるとのご指摘を頂きました。
どうやらいくつかの**破壊的変更**があったようです。
[参考書籍の第2版](https://book.impress.co.jp/books/1122101167)が今年の9月に出版されたので、内容を更新したいと思います。

## 参考
* [pandasライブラリ活用入門[第2版] データクリーニング/前処理など一連の分析プロセスをマスター!](https://book.impress.co.jp/books/1122101167)
  * pandas単体で完結する第1, 2, 4, 5, 6, 8章のコードを記載します。
* [API reference — pandas 2.1.4 documentation](https://pandas.pydata.org/docs/reference/index.html)
* [API reference — Polars documentation](https://pola-rs.github.io/polars/py-polars/html/reference/index.html)

## 前回記事との違い
1. 前回は[原著のGitHub](https://github.com/chendaniely/pandas_for_everyone)に沿ってコードを記載しましたが、今回は書籍の内容に沿ってコードを書いていきます。
書籍を買われた方にとって、より見やすい構成になるかと思います。

2. すべてのコードブロックに```import pandas/polars```を記入するので、pandasとpolarsの句は分けないことにします。

3. ```DataFrame```の表示では、```print```ではなく```display```を使います。

## ライブラリ
* ```pandas 2.1.4```
* ```polars 0.19.19```
* ```numpy 1.26.2```
* ```seaborn 0.13.0```

In [ ]:
import polars as pl
#pl.Config.set_tbl_cols(-1) # 列表示を省略しない
#pl.Config.set_tbl_rows(-1) # 行の表示を省略しない

import pandas as pd
#pd.set_option("display.max_rows", 500)
#pd.set_option("display.min_rows", 500)

# 第1部　基本的な使い方
## 第1章 DataFrame
### 1.2 最初のデータセットをロードする

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("DataFrameの型", type(df))
display("DataFrameの(行数, 列数)", df.shape)
display("DataFrameの列名", df.columns)
display("各列のデータ型", df.dtypes)
display("DataFrameの基本情報", df.info())


In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("DataFrameの型", type(df))
display("DataFrameの(行数, 列数)", df.shape)
display("DataFrameの列名", df.columns)
display("各列のデータ型", df.dtypes)
display("DataFrameの基本情報", df.describe().transpose(include_header = True))


### 1.3 列、行、セルを見る

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("先頭の5行を表示する", df.head())

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("先頭の5行を表示する", df.head())

#### 1.3.1 名前で列を絞り込む

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
country_df = df["country"]
display("最初の5個の値を表示する", country_df.head())
display("最後の5個の値を表示する", country_df.tail())

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
country_df = df["country"]
display("最初の5個の値を表示する", country_df.head(n = 5))
display("最後の5個の値を表示する", country_df.tail(n = 5))

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
subset = df[["country", "continent", "year"]]
display("国と大陸と年を見る", subset)

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
subset = df[["country", "continent", "year"]]
display("国と大陸と年を見る", subset)

#### 1.3.1.1 1個の値で返されるDataFrameやSeries

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
country_df = df["country"]
display("[] で国の列を抽出した場合", country_df.head())
display("データ型", type(country_df))
country_df_list = df[["country"]]
display("[[]] で国の列を抽出した場合", country_df_list.head())
display("データ型", type(country_df_list))

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
country_df = df["country"]
display("[] で国の列を抽出した場合", country_df.head(n = 5))
display("データ型", type(country_df))
country_df_list = df[["country"]]
display("[[]] で国の列を抽出した場合", country_df_list.head())
display("データ型", type(country_df_list))

#### 1.3.2 複数行の抽出

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("最初の行を取る", df.loc[0])
last_row_index = df.shape[0] - 1
display("最後の行を取る", df.loc[last_row_index])

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("最初の行を取る", df.slice( offset = 0, length = 1))
last_row_index = df.shape[0] - 1
display("最後の行を取る", df.slice( offset = last_row_index, length = 1))

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("locで最初の行を取得した場合の型", type(df.loc[0]))
display("head()で最初の行を取得した場合の型", type(df.head(n = 1)))

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("sliceで最初の行を取得した場合の型", type(df.slice( offset = 0, length = 1)))
display("head()で最初の行を取得した場合の型", type(df.head(n = 1)))

#### 1.3.2.2 複数行の抽出

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("複数行を抽出する", df.loc[[0, 99, 999]])

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("複数行を抽出する", df[ [0, 99, 999] ])

#### 1.3.3 行番号による複数行の抽出: iloc[]

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("2番目の行を見る", df.iloc[1])
display("100番目の行を見る", df.iloc[99])
display("-1 を使って最後の行を見る", df.iloc[-1])
display("第1行と第100行と第1000行を選択する", df.iloc[[0, 99, 999]])

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("2番目の行を見る", df[ 1 ])
display("100番目の行を見る", df[ 99 ])
display("-1 を使って最後の行を見る", df[ -1 ])
display("第1行と第100行と第1000行を選択する", df[ [0, 99, 999] ])

#### 1.3.4 組み合わせで絞り込む
#### 1.3.4.1 複数列の抽出

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("locと列名で列を絞り込む", df.loc[:, ["year", "pop"]].head(n = 2))
display("ilocと列番号で列を絞り込む", df.iloc[:, [2, 4, -1]].head(n = 2))

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("[]と列名で列を絞り込む", df[:, ["year", "pop"]].head(n = 2))
display("[]と列番号で列を絞り込む", df[:, [2, 4, -1]].head(n = 2))

#### 1.3.4.2 範囲による複数列の抽出

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("第0列から第4列までを絞り込む", df.iloc[:, list(range(5))].head(n = 2))
display("第3列から第5列までを絞り込む", df.iloc[:, list(range(3, 6))].head(n = 2))
display("第0列から第5列まで1つおきに絞り込む", df.iloc[:, list(range(0, 6, 2))].head(n = 2))

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("第0列から第4列までを絞り込む", df[:, list(range(5))].head(n = 2))
display("第3列から第5列までを絞り込む", df[:, list(range(3, 6))].head(n = 2))
display("第0列から第5列まで1つおきに絞り込む", df[:, list(range(0, 6, 2))].head(n = 2))

#### 1.3.4.3 スライスによる複数列の絞り込み

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("最初の3列を絞り込む", df.iloc[:, list(range(3))].head(n = 2))
display("最初の3列をスライス", df.iloc[:, :3].head(n = 2))
display("第3列から第5列までを絞り込む", df.iloc[:, list(range(3, 6))].head(n = 2))
display("第3列から第5列までをスライス", df.iloc[:, 3:6].head(n = 2))
display("第0列から第5列まで1つおきに絞り込む", df.iloc[:, list(range(0, 6, 2))].head(n = 2))
display("第0列から第5列まで1つおきにスライス", df.iloc[:, 0:6:2].head(n = 2))

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("最初の3列を絞り込む", df[:, list(range(5))].head(n = 2))
display("最初の3列をスライス", df[:, :3].head(n = 2))
display("第3列から第5列までを絞り込む", df[:, list(range(3, 6))].head(n = 2))
display("第3列から第5列までをスライス", df[:, 3:6].head(n = 2))
display("第0列から第5列まで1つおきに絞り込む", df[:, list(range(0, 6, 2))].head(n = 2))
display("第0列から第5列まで1つおきにスライス", df[:, 0:6:2].head(n = 2))

#### 1.3.5 列と行の抽出

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("locを用いて、第43行のcountry列の要素を抽出する", df.loc[42, "country"])
display("ilocを用いて、第43行の第0列の要素を抽出する", df.iloc[42, 0])

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("第43行のcountry列の要素を抽出する", df[42, "country"])
display("第43行の第0列の要素を抽出する", df[42, 0])

#### 1.3.5.1 複数行、複数列の抽出

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("第1行、第100行、第1000行を、第1列、第4列、第6列から切り出す", df.iloc[[0, 99, 999], [0, 3, 5]])
display("先の抽出を、列名で実行する場合", df.loc[[0, 99, 999], ["country", "lifeExp", "gdpPercap"]])

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("第1行、第100行、第1000行を、第1列、第4列、第6列から切り出す", df[[0, 99, 999], [0, 3, 5]])
display("先の抽出を、列名で実行する場合", df[[0, 99, 999], ["country", "lifeExp", "gdpPercap"]])

### 1.4 グループ分けと集約の計算
疑問
1. 各年度で予期された余命(```lifeExp```)の平均値は、いくつなのだろうか。人口(```pop```)やGDP(```gdpPercap```)の平均値も知りたい
2. データを大陸(```continent```)ごとに分け、それぞれに同じ計算を実行したら、どういう結果になるか。
3. それぞれの大陸のリストに、どれだけの国(```country```)が入っているか。

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("グループ分けの前にデータを眺めてみる", df)

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("グループ分けの前にデータを眺めてみる", df)

#### 1.4.1 グループごとの平均値

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
# 各年度で予想された余命の平均値は?
# この疑問に答えるには、
# 1. まずデータを年度によって分割し、
# 2. 毎年の"lifeExp"列を取り出し、
# 3. その平均値を計算する
display("各年度で予想された余命の平均値", df.groupby("year")["lifeExp"].mean())

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
# 各年度で予想された余命の平均値は?
# この疑問に答えるには、
# 1. まずデータを年度によって分割し、
# 2. 毎年の"lifeExp"列を取り出し、
# 3. その平均値を計算する
display("各年度で予想された余命の平均値", df.group_by("year").agg( pl.col("lifeExp").mean() ).sort(by = "year"))

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
# 各年度、大陸別の予想された余命の平均値は?
display("各年度、大陸別で予想された余命の平均値",
        df\
                .groupby(["year", "continent"])[["lifeExp", "gdpPercap"]]\
                        .mean())

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
# 各年度、大陸別の予想された余命の平均値は?
display("各年度、大陸別で予想された余命の平均値",
        df\
    .group_by(["year", "continent"])\
    .agg([ pl.col("lifeExp").mean(), pl.mean("gdpPercap")])\
    .sort(by = "year"))

#### 1.4.2 グループごとの度数/頻度

In [ ]:
import pandas as pd
df = pd.read_csv("../data/gapminder.tsv", sep = "\t")
display("各大陸に入っている国の数", df.groupby("continent")["country"].nunique())

In [ ]:
import polars as pl
df = pl.read_csv("../data/gapminder.tsv", separator = "\t")
display("各大陸に入っている国の数",
        df\
            .group_by(by = "continent")\
                .agg( pl.col("country").n_unique() )\
                    .sort(by = "continent"))